In [16]:
import numpy as np
import polar_code
import scipy
from scipy import sparse 
N=1024
K=512
cd=polar_code.coding(N)
itr_num=np.log2(N).astype(int)

In [17]:
D=np.ones((K,N),dtype=int)
for i in cd.frozen_bits:
  tmp=np.count_nonzero(cd.info_bits<i)
  D[tmp:,i]=0

print(len(cd.info_bits))

#change D into row echelon form
res=np.ones((K,N),dtype=int)
count=0
for i in range(K):
  for j in range(N):
    if D[i,j]==0:
      res[:,count]=D[:,j]#insert to res
      D[:,j]=2
      count+=1

print(count)


  
np.savetxt("D",res,fmt='%i')

512
512


In [18]:
print(res)

[[0 0 0 ... 1 1 1]
 [0 0 0 ... 1 1 1]
 [0 0 0 ... 1 1 1]
 ...
 [0 0 0 ... 1 1 1]
 [0 0 0 ... 1 1 1]
 [0 0 0 ... 1 1 1]]


In [19]:
print("static frozen bit number is")
sfnum=np.count_nonzero(res[0]==0)
print(sfnum)

print("dynamic frozen bit number is")
dfnum=np.count_nonzero(res[K-1]==0)-sfnum
print(dfnum)

print("informatino bit num is")
print(N-sfnum-dfnum)

static frozen bit number is
127
dynamic frozen bit number is
385
informatino bit num is
512


In [20]:
P=make_H(itr_num) #poalr generator matrix
GG=res@P

In [21]:
np.savetxt("GG",GG,fmt='%i')

In [22]:
def interleave(N):

    interleaver_sequence=np.arange(N)
    np.random.shuffle(interleaver_sequence)
    return interleaver_sequence

def generate_regular_H(N,K,Wc,Wr):
  '''
  #generate regular parity check matrix
  #-----------
  #Wr : row weight
  #Wc : column weight
  #N : length of codeword 
  '''

  if N*Wc%Wr!=0:
    print("constant err")
    exit()

  #generate sub_H matrix(Wc=1)
  sub_H=np.zeros(((N-K)//Wc,N),dtype=int)
  for i in range((N-K)//Wc):
      sub_H[i][Wr*i:Wr*(i+1)]=1

  H=sub_H

  #generate other sub_H matrix(Wc=1)
  for i in range(Wc+1):
    sub_H2=sub_H[:,interleave(N)]
    H=np.concatenate((H,sub_H2))
  
  H=H[:K,:]

  return H 

In [23]:
def tensordot(A):
  tmp0=np.zeros((A.shape[0],A.shape[1]),dtype=np.int)
  tmp1=np.append(A,tmp0,axis=1)
  #print(tmp1)
  tmp2=np.append(A,A,axis=1)
  #print(tmp2)
  tmp3=np.append(tmp1,tmp2,axis=0)
  #print(tmp3)
  return tmp3

def make_H(itr_num):
  G2=np.array([[1,0],[1,1]],dtype=np.int)
  Gres=G2
  for _ in range(itr_num-1):
    #print(i)
    Gres=tensordot(Gres)
  return Gres

In [24]:
def binaryproduct(X, Y):
    """Compute a matrix-matrix / vector product in Z/2Z."""
    A = X.dot(Y)
    try:
      A = A.toarray()
    except AttributeError:
      pass
    return A % 2

def gaussjordan(X, change=0):
    """Compute the binary row reduced echelon form of X.
    Parameters
    ----------
    X: array (m, n)
    change : boolean (default, False). If True returns the inverse transform
    Returns
    -------
    if `change` == 'True':
        A: array (m, n). row reduced form of X.
        P: tranformations applied to the identity
    else:
        A: array (m, n). row reduced form of X.
    """
    A = np.copy(X)
    m, n = A.shape

    if change:
      P = np.identity(m).astype(int)

    pivot_old = -1
    for j in range(n):
      filtre_down = A[pivot_old+1:m, j]
      pivot = np.argmax(filtre_down)+pivot_old+1

      if A[pivot, j]:
        pivot_old += 1
        if pivot_old != pivot:
          aux = np.copy(A[pivot, :])
          A[pivot, :] = A[pivot_old, :]
          A[pivot_old, :] = aux
          if change:
            aux = np.copy(P[pivot, :])
            P[pivot, :] = P[pivot_old, :]
            P[pivot_old, :] = aux

        for i in range(m):
          if i != pivot_old and A[i, j]:
            if change:
              P[i, :] = abs(P[i, :]-P[pivot_old, :])
            A[i, :] = abs(A[i, :]-A[pivot_old, :])

      if pivot_old == m-1:
        break

    if change:
      return A, P
    return A

In [25]:
def HtotG(H,sparse=False):
    """Return the generating coding matrix G given the LDPC matrix H.
    Parameters
    ----------
    H: array (n_equations, n_code). Parity check matrix of an LDPC code with
        code length `n_code` and `n_equations` number of equations.
    sparse: (boolean, default True): if `True`, scipy.sparse format is used
        to speed up computation.
    Returns
    -------
    G.T: array (n_bits, n_code). Transposed coding matrix.
    """

    if type(H) == scipy.sparse.csr_matrix:
      H = H.toarray()
    n_equations, n_code = H.shape

    # DOUBLE GAUSS-JORDAN:

    Href_colonnes, tQ = gaussjordan(H.T, 1)

    Href_diag = gaussjordan(np.transpose(Href_colonnes))

    Q = tQ.T

    n_bits = n_code - Href_diag.sum()

    Y = np.zeros(shape=(n_code, n_bits)).astype(int)
    Y[n_code - n_bits:, :] = np.identity(n_bits)

    if sparse:
      Q = scipy.sparse.csr_matrix(Q)
      Y = scipy.sparse.csr_matrix(Y)

    tG = binaryproduct(Q, Y)

    return tG

In [26]:
print(res)

[[0 0 0 ... 1 1 1]
 [0 0 0 ... 1 1 1]
 [0 0 0 ... 1 1 1]
 ...
 [0 0 0 ... 1 1 1]
 [0 0 0 ... 1 1 1]
 [0 0 0 ... 1 1 1]]


In [33]:
def fill_one_row(D,count):
  for i in reversed(range(D.shape[0])):
    for j in range(D.shape[1]):
      if D[i,j]==1:
        res=D[:,j]
        D=np.delete(D,j,axis=1)
        count-=1
        return D,res,count


In [35]:
H=generate_regular_H(N,K,3,6)
G=make_H(itr_num)
A=G@H.T%2
tG=HtotG(A.T)
G=tG.T
D=G@P%2
Dnew=np.zeros((K,N))
#Dをrow echelon formにする
count=N-1
while count>0:
  D,res,count=fill_one_row(D,count)
  Dnew[:,count]=res

np.savetxt("DD",Dnew,fmt='%i')

In [34]:
count=0
while True:
  H=generate_regular_H(N,K,3,6)
  G=make_H(itr_num)
  A=G@H.T%2
  tG=HtotG(A.T)
  G=tG.T
  D=G@P
  Dnew=np.zeros((K,N))
  #Dをrow echelon formにする
  tmp=N-1
  while count>0:
    D,res,tmp=fill_one_row(D,tmp)
    Dnew[:,tmp]=res
    
  if np.any((D-res)==-1)==False:
    np.savetxt("G",G,fmt='%i')
    break
  count+=1
  print(count)




1


KeyboardInterrupt: 

In [15]:
print(tG)
np.linalg.matrix_rank(tG)
print(np.any((tG.T@A%2)!=0))

[[0 0 0 ... 0 0 0]
 [1 1 0 ... 0 1 0]
 [0 1 0 ... 1 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]
False


In [2]:
CRC_polynomial =np.array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1])
CRC_len=len(CRC_polynomial)

In [3]:
def left_cyclic(data,polynomial,memory):
  tmp=np.zeros(len(memory)+1,dtype=int)
  tmp[0]=data
  tmp[1:]=memory
  res=np.sum(tmp*polynomial)%2
  memory=tmp[:len(memory)]
  return res,memory

def cyclic(data,polynomial,memory):

  tmp=-1*np.ones(len(memory)+1,dtype=int)
  tmp[len(tmp)-1]=data
  pre_data=memory[0]
  tmp[:len(tmp)-1]=memory

  for i in range(len(polynomial)-1):
    if polynomial[i]==1:
      tmp[i]=(pre_data+tmp[i+1])%2
    else:
      tmp[i]=tmp[i+1]

  #from IPython.core.debugger import Pdb; Pdb().set_trace()
  memory=tmp[:len(memory)]

  return memory

In [4]:
CRC_polynomial =np.array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1])
info_len=15
information=np.random.randint(0,2,info_len)
#print(information)

def CRC_gen(information,parity,polynomial):
  CRC_info=np.zeros(len(information)+len(parity),dtype='int')
  CRC_info[:len(information)]=information
  CRC_info[len(information):]=parity

  memory=np.zeros(CRC_len-1,dtype='int')
  CRC_info[:len(information)]=information
  for i in range(len(CRC_info)):
    memory=cyclic(CRC_info[i],polynomial,memory)
  #print(len(memory))
  CRC_info[info_len:]=memory

  return CRC_info,np.all(memory==0)

parity=np.zeros(len(CRC_polynomial)-1)
CRC_info,check=CRC_gen(information,parity,CRC_polynomial)
print(information)
print(CRC_info)
parity=CRC_info[len(information):]
information=CRC_info[:len(information)]
CRC_info,check=CRC_gen(information,parity,CRC_polynomial)
print(check)

[1 0 0 1 0 1 1 0 0 0 0 0 1 0 1]
[1 0 0 1 0 1 1 0 0 0 0 0 1 0 1 0 1 0 0 0 1 1 1 1 0 0 1 0 0 0 0]
True


In [5]:
def CRC_check(self,information):
  memory=np.zeros(CRC_len-1)
  for i in range(len(CRC_info)):
    memory=self.right_cyclic(CRC_info[i],CRC_polynomial,memory)
  
  if np.all(memory==0):
    return True
  
  else:
    return False

In [ ]:

CRC_info=np.zeros(K+CRC_len-1)
memory=np.zeros(CRC_len-1,dtype=int)
for i in range(K):
  CRC_info[i],memory=left_cyclic(information[i],CRC_polynomial,memory)

CRC_info[self.K:]=memory

print(CRC_check(CRC_info))
if CRC_check(CRC_info)==False:
  print("CRC_err")

print(CRC_info)